In [1]:
# Connect to Spark by creating a Spark session
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession\
    .builder\
    .appName("Dataframe")\
    .getOrCreate()

sc = spark.sparkContext

In [2]:
data1 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William'},
         'Sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}}

df1_pd = pd.DataFrame(data1, columns=data1.keys())

In [3]:
df1 = spark.createDataFrame(df1_pd)
df1.show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



In [4]:
df1.createOrReplaceTempView('result')

In [5]:
#Template Column List
columnsList = ['col1','col2','col3','col4','col5']

#Column Mapping
columnMapping = {
    'col1' : 'PassengerId',
    'col2' : 'Sex',
    'col3' : 'Survived'
}

In [6]:
#Genearate sql based on Template and Column Mapping
sql = 'select '
result_df = 'result'
for column in columnsList:
    if column not in columnMapping:
        sql += 'null as {},'.format(column)
    else:
        sql += '{}.{} as {},'.format(result_df,columnMapping[column],column)
        
sql = sql[:-1]
sql += ' from {}'.format(result_df)
print(sql)

select result.PassengerId as col1,result.Sex as col2,result.Survived as col3,null as col4,null as col5 from result


In [7]:
final_result = spark.sql(sql)

In [8]:
#Missing columns in mapping are populated as null
final_result.show()

+----+------+----+----+----+
|col1|  col2|col3|col4|col5|
+----+------+----+----+----+
|   1|  male|   0|null|null|
|   2|female|   1|null|null|
|   3|female|   1|null|null|
|   4|female|   1|null|null|
|   5|  male|   0|null|null|
+----+------+----+----+----+

